In [ ]:
import cv2
import pytesseract
from pytesseract import Output
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt
from ocrutils.imageops import is_a_picture, prepare_image

Carregando imagem

In [ ]:
path_data = Path.cwd().joinpath('data','wordsearch.png')
img = cv2.imread(str(path_data.resolve()))
plt.imshow(img)

config tesseract

In [ ]:
tessdata_dir = Path.cwd().joinpath('tessdata')
whitelist = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789áàâãéêíóôõúüçÁÀÂÃÉÊÍÓÔÕÚÜÇ"
blacklist = " "
config_shotScream = f'--psm 11 --tessdata-dir {tessdata_dir} -c tessedit_char_whitelist={whitelist} -c tessedit_char_blacklist={blacklist}'
config_picture = f'--psm 11 --tessdata-dir {tessdata_dir} -c tessedit_char_whitelist={whitelist} -c tessedit_char_blacklist={blacklist}'


Gray + threshold

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, img = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)

Reconhecimento das regiões

In [ ]:
if (not is_a_picture(img)):
    img_data = pytesseract.image_to_data(img, config=config_shotScream, lang ='por', output_type=Output.DICT)
    color = ()
else:
    img = prepare_image(img)
    img_data = pytesseract.image_to_data(img, config=config_picture, lang ='por', output_type=Output.DICT)

In [ ]:
# Retorna uma lista com os indices das possiveis palavras detectadas no dicionario
def detect_words(words: list, data : dict) -> list:
    chaves = list(data.keys())
    size = len(data[chaves[0]])
    for i in range(size):
        if len(data['text'][i]) > 4:
            words.append(i)
    return words

def detect_words_area(data : dict):
    words = []
    words = detect_words(words = words, data = data)

def detect_letters_area():
    print('carro')

detect_words_area(img_data)

ADICIONAL
CARTO
ROUBARAM
ALERTA
CONTAS
GARANTIA
NEGOCIAR
SEGURA
CRDITO
GOLPE
NUBANK
SEGURO
APOSENTADOS
DEFESAS
GRAVAES
NUTAG
ATENDIMENTO
DIRETO
JUNTAS
PARCELA
TESOURO
BLOQUEAR
DBITO
PARENTAL
VALOR
BLOQUEIOS
ESCURO
LIGAES
PORTABILIDADE
VIDAS
CANAL
FALSAS
LIMITES
PROTEO
VIRTUAL
